# data load & 전처리

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

In [2]:
df_train = pd.read_csv('./data/act_train.csv', parse_dates=['date'])
df_people = pd.read_csv('./data/people.csv', parse_dates=['date'])
df_merge = pd.merge(df_train, df_people, on='people_id')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

In [ ]:
df_date_x = df_merge['date_x'].copy()
df_date_y = df_merge['date_y'].copy()

df_merge.drop(['date_x', 'date_y'], axis=1, inplace=True)

In [ ]:
df_merge = df_merge.replace('type ', '', regex=True).apply(pd.to_numeric, errors='ignore')

In [ ]:
df_merge.insert(loc=2 ,column='date_x', value=df_date_x)
df_merge.insert(loc=18 ,column='date_y', value=df_date_y)

#### test data

In [ ]:
# df_test = pd.read_csv('./data/act_test.csv', parse_dates=['date'])
# df_merge_test = pd.merge(df_test, df_people, how='inner', on='people_id')
# df_date_x_test = df_merge['date_x'].copy()
# df_date_y_test = df_merge['date_y'].copy()

# df_merge_test.drop(['date_x', 'date_y'], axis=1, inplace=True)

# df_merge_test = df_merge_test.replace('type ', '', regex=True).apply(pd.to_numeric, errors='ignore')

# df_merge_test.insert(loc=2 ,column='date_x', value=df_date_x_test)
# df_merge_test.insert(loc=18 ,column='date_y', value=df_date_y_test)

In [ ]:
del df_train, df_people, df_date_x, df_date_y
# del df_test, df_date_x_test, df_date_y_test

In [ ]:
# df_merge.to_csv("train_merge.csv")

In [ ]:
# df_merge = pd.read_csv('./train_merge.csv', parse_dates=['date_x','date_y'])

In [ ]:
# df_merge.dtypes

# EDA

In [ ]:
for d in ['date_x', 'date_y']:
    print('Start of ' + d + ': ' + str(df_merge[d].min().date()))
    print('  End of ' + d + ': ' + str(df_merge[d].max().date()))
    print('Range of ' + d + ': ' + str(df_merge[d].max() - df_merge[d].min()) + '\n')

In [ ]:
df_merge['date_y'].min().date()

In [ ]:
df_merge['date_x'].min().date()

In [ ]:
df_sample = df_merge.sample(frac = 0.1).copy()

In [ ]:
a = df_sample.groupby('people_id')
a.head()